# Building Accurate Retrieval Method

LLM is pulling context from vector db. This allows the 

Document Loader

load documents from list of filepaths
get metadata from each pdf for more context

In [1]:
# Get PDF files
import os
import requests

list_of_pdf_docs = [
    "https://www.faa.gov/sites/faa.gov/files/regulations_policies/handbooks_manuals/aviation/airplane_handbook/00_afh_full.pdf",
    "https://www.faa.gov/sites/faa.gov/files/regulations_policies/handbooks_manuals/aviation/FAA-H-8083-15B.pdf",
    "https://www.faa.gov/regulations_policies/handbooks_manuals/aviation/faa-h-8083-25c.pdf",
    "https://www.faa.gov/sites/faa.gov/files/2022-06/risk_management_handbook_2A.pdf",
    "https://www.faa.gov/sites/faa.gov/files/regulations_policies/handbooks_manuals/aviation/FAA-H-8083-1.pdf",
]

In [ ]:
### Document Loader

from langchain_community.document_loaders import PyMuPDFLoader

